Based on https://github.com/huggingface/transformers/blob/3763f8944dc3fef8afb0c525a2ced8a04889c14f/examples/extract_features.py

Needs to be cleaned up :)

In [0]:
!pip install pytorch_pretrained_bert
!pip install seqeval

In [0]:
import torch
import urllib
import pandas as pd
from pytorch_pretrained_bert import BertForTokenClassification, BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import collections
import json

In [0]:
MAX_LEN = 50
batch_size = 32
TRAIN_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AD7GEDPOUJFOQS3HTDRWMOS6KZP62'
DEV_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/dev-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AD7GEDNCXBQCYYC5ZKLNIWC6KZP6Y'

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = BertModel.from_pretrained('bert-base-uncased')
model.cuda()

In [0]:
def get_comments(filename, url=True):
    if url:
        comments = []
        with urllib.request.urlopen(filename) as f:
            for line in f:
                if line.startswith(b'#'):
                    comments.append(line.decode("utf-8"))
                else:
                    break
        return comments
    with open(filename, 'r', encoding='utf8') as f:
        commentiter = takewhile(lambda s: s.startswith('#'), f)
        comments = list(commentiter)
    return comments

In [0]:
comments = get_comments(TRAIN_URL)
train_df = pd.read_csv(TRAIN_URL, sep='\t', skiprows=len(comments), quoting=3)
train_input = train_df.groupby('sent_id')['token'].apply(list).to_frame()

In [0]:
# Adapted from https://github.com/huggingface/transformers/blob/3763f8944dc3fef8afb0c525a2ced8a04889c14f/examples/extract_features.py

class InputExample(object):

    def __init__(self, unique_id, text):
        self.unique_id = unique_id
        self.text = text


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, sent_id, tokens, input_ids, input_mask, input_type_ids):
        self.sent_id = sent_id
        self.tokens = tokens
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.input_type_ids = input_type_ids


def convert_sentences_to_features(sentences, seq_length, tokenizer):
    """Loads a data file into a list of `InputFeature`s."""

    features = []
    for (idx, tok_list) in enumerate(sentences):
        tok_list = [str(tok).lower() for tok in tok_list]
        sentence = ' '.join(tok_list)
        tokens = tokenizer.tokenize(sentence)
        idx += 1  # Our sentence IDs are 1-indexed

        # Taking into account [CLS] and [SEP] here
        if len(tokens) + 2 > seq_length:
            print('Sentence is too long', len(tokens), idx)
            print(sentence)
            print(tokens)
            tokens = tokens[0:(seq_length - 2)]

        input_type_ids = []  # TODO do we need these?
        tokens.insert(0, '[CLS]')
        for token in tokens:
            input_type_ids.append(0)
        tokens.append('[SEP]')
        input_type_ids.append(0)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < seq_length:
            input_ids.append(0)
            input_mask.append(0)
            input_type_ids.append(0)

        assert len(input_ids) == seq_length
        assert len(input_mask) == seq_length
        assert len(input_type_ids) == seq_length

        features.append(
            InputFeatures(
                sent_id=idx,
                tokens=tokens,
                input_ids=input_ids,
                input_mask=input_mask,
                input_type_ids=input_type_ids))
    return features


In [0]:
sentences = train_input['token'].tolist()
features = convert_sentences_to_features(sentences, MAX_LEN + 2, tokenizer)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_example_index)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [0]:
output_file = 'bert.txt'
layer_indexes = [-1]

model.eval()
with open(output_file, "w", encoding='utf-8') as writer:
    for input_ids, input_mask, example_indices in train_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)

        all_encoder_layers, _ = model(input_ids, token_type_ids=None, attention_mask=input_mask)
        all_encoder_layers = all_encoder_layers

        for b, example_index in enumerate(example_indices):
            feature = features[example_index.item()]
            sent_id = int(feature.sent_id)
            if (sent_id % 1000 == 0):
                print(sent_id)
            output_json = collections.OrderedDict()
            output_json["linex_index"] = sent_id
            all_out_features = []
            for (i, token) in enumerate(feature.tokens):
                all_layers = []
                for (j, layer_index) in enumerate(layer_indexes):
                    layer_output = all_encoder_layers[int(layer_index)].detach().cpu().numpy()
                    layer_output = layer_output[b]
                    layers = collections.OrderedDict()
                    layers["index"] = layer_index
                    layers["values"] = [
                        round(x.item(), 6) for x in layer_output[i]
                    ]
                    all_layers.append(layers)
                out_features = collections.OrderedDict()
                out_features["token"] = token
                out_features["layers"] = all_layers
                all_out_features.append(out_features)
            output_json["features"] = all_out_features
            writer.write(json.dumps(output_json) + "\n")

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

In [0]:
!cp bert.txt "/content/gdrive/My Drive"

In [66]:
!head "/content/gdrive/My Drive/bert.txt"

{"linex_index": 1, "features": [{"token": "[CLS]", "layers": [{"index": -1, "values": [-0.186671, -0.308374, 0.233253, 0.138232, -0.519236, -0.040894, 0.396187, 0.333497, -0.260176, -0.446584, 0.123103, -0.143987, -0.279792, 0.278411, 0.220393, 0.111958, -0.168582, 0.516053, 0.314872, -0.158025, 0.244905, -0.289928, 0.192974, -0.057787, 0.047926, -0.143381, 0.107548, -0.364693, -0.165039, 0.071327, -0.126826, 0.388706, -0.477373, -0.184424, 0.052022, -0.150257, -0.023197, 0.034768, 0.324394, 0.092805, -0.288317, -0.33604, 0.134116, 0.2551, -0.373391, -0.11548, -2.858938, 0.058526, -0.362582, -0.11874, 0.124834, -0.133669, 0.186649, 0.198359, 0.05365, 0.206357, -0.069027, 0.36757, -0.30094, -0.099594, 0.527053, -0.310872, -0.292744, -0.23061, -0.232332, 0.104664, -0.019937, 0.272095, -0.121486, 0.619773, -0.428696, -0.196438, 0.320799, 0.162844, -0.290792, -0.549824, 0.014297, 0.260864, 0.035359, -0.261238, -0.086188, 0.304101, -0.115047, -0.32693, 0.271517, 0.363755, -0.001835, -0.3361